# Sample Notebook for running regression on HCal Insert.

In [ ]:
#Python/Data
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.colors as mcolors
import numpy as np
import h5py as h5

#ML
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split
import tensorflow as tf

from sklearn.preprocessing import StandardScaler


In [ ]:
filename = '/home/miguel/generate_data/to_hdf5/insert_reco_pi-_60GeV_theta_20.83deg_10000events.edm4hep.hdf5'#insert_reco_pi-_50GeV_theta_2.83deg.edm4hep.hdf5'#rec_pionplus_10_80_HCALonly.hdf5'#rec_pipl1000_0-100GeV.hdf5'
h5_file = h5.File(filename,'r')
print(list(h5_file.keys()))
images = h5_file['hcal']
truth = h5_file['mc']

print(images.shape)
print(truth.shape)
#print(images.chunks) # [0] should be = batch_size

In [ ]:
N_Events = 10000
#X = images[0:N_Events]
#Y = truth[0:N_Events,8,0]

X = images[0:N_Events]
Y = truth[0:N_Events,8,0]

print(np.shape(X))
print(np.shape(Y))
plt.hist(Y)
plt.title("MC Energy")
_ = plt.xlabel("Energy [GeV]")

In [ ]:
#basicreco = np.sum(np.nan_to_num(X[:,0,:]),axis=1)
#print(basicreco)
#print(basicreco.shape)
#print(Y.shape)
#plt.scatter(Y,basicreco,alpha=0.3)
#plt.xlim(-10,110)


In [ ]:
#data = []
#data.insert(0,np.ravel(X[:,0,:]))
#temp = np.ravel(X[:,0,:])
#plt.hist(data[0][data[0]!=0],bins=100)
#plt.show()

In [ ]:
for i in range(X.shape[1]):
    E_mean = np.nanmean(X[:,i,:])
    E_stdev = np.nanstd(X[:,i,:])
    X[:,i,:] = (X[:,i,:]-E_mean)/E_stdev

X = np.nan_to_num(X)
Y = np.nan_to_num(Y)

# Important: Need to normalize the input `X` data. This is done as (X-Mean)/StdDev. Recomend doing this and adding ECal information in a separate notebook or C file

In [ ]:
cm = plt.cm.get_cmap('plasma')
cell_vars = ["Energy","Cell X","Cell Y","Cell Z"]

data=[]
fig = plt.figure(figsize=(18,9))
#plt.subplots_adjust(left=None, bottom=1, right=None, top=1.5, wspace=None, hspace=None)
#print(images.shape[1])
for i in range(X.shape[1]):
    ax = plt.subplot(2, 3, i+1)
#    data.insert(i,np.ravel(images[0:10000,i,:]))
    data.insert(i,np.ravel(X[0:10000,i,:]))

    data[i] = data[i][data[i]!=0]
    plt.hist(data[i],color=cm(i/X.shape[1]),bins=100)
    plt.title("%s"%(cell_vars[i]),fontsize=20)
    plt.suptitle("Non Normalized Cell Input Data",fontsize=25)
plt.savefig("Normalized_Cell_Data.pdf")

In [ ]:
(X_train, X_val, X_test,
Y_train, Y_val, Y_test) = data_split(X, Y, val=0.2, test=0.3,shuffle=True)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

earlystopping = EarlyStopping(patience=10,
                              #verbose=verbose,
                              restore_best_weights=True)

In [ ]:
learning_rate = 1e-6
Phi_sizes, F_sizes = (100, 100, 128), (100, 100, 100)
#Phi_sizes, F_sizes = (50, 50, 64), (50, 50, 64)
#Phi_sizes, F_sizes = (25, 25, 32), (25, 24, 32)

output_act, output_dim = 'linear', 1
loss = 'mse' #mean-squared error
pfn = PFN(input_dim=X.shape[-1], Phi_sizes=Phi_sizes, F_sizes=F_sizes, 
          output_act=output_act, output_dim=output_dim, loss=loss, 
          optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
          #callbacks=[earlystopping],
          patience=10,
          F_dropouts=0.01)

In [ ]:
the_fit = pfn.fit(X_train, Y_train,
                  epochs=400, #200 nominal
                  batch_size=400,
                  validation_data=(X_val, Y_val),
                  verbose=1)

In [ ]:
pfn.layers
pfn.save("energy_regression.h5")
mypreds = pfn.predict(X_test,batch_size=400)

In [ ]:
fig = plt.figure(figsize=(28,10))
ax = plt.subplot(1, 2, 1)
plt.scatter(Y_test,mypreds,alpha=0.3)
plt.xlabel("Y Test [GeV]",fontsize=22)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tick_params(direction='in',right=True,top=True,length=10)
#plt.ylim(-0.01,100.01)
plt.ylim(-10,110)
plt.xlim(-10,110)

plt.ylabel("Y Pred [GeV]",fontsize=22)
_ = plt.title("Prediction vs. Test",fontsize=26)
#FIXME: find indecies in mypreds where mypreds 50

ax = plt.subplot(1, 2, 2)
plt.plot(the_fit.history['loss'])
plt.plot(the_fit.history['val_loss'])
plt.title('Model Loss vs. Epoch',fontsize=26)
plt.text(0.73,0.8," Rate = %1.7f"%(learning_rate),
         transform=ax.transAxes,fontsize=20)
plt.ylabel('Mean-Squared Error',fontsize=22)
plt.yscale('log')
plt.xlabel('epoch',fontsize=22)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tick_params(direction='in',right=True,top=True,length=10)
plt.tick_params(direction='in',right=True,top=True,which='minor')
#plt.xlim([-1,201])
plt.ylim([1,10000])
plt.legend(['train', 'validation'], loc='upper right',fontsize=22)
plt.savefig("Prediction_Test.pdf")

In [ ]:
print(Y_test.shape)
print(Y_test[:])
print(np.array(mypreds.flatten()))
print(X_test[:,0,:].shape)

print(np.sum(X_test[:,0,:],axis=1).shape)

basic = np.sum(X_test[:,0,:],axis=1)

In [ ]:
from scipy import stats
import pandas as pd

df = {}
df['gen'] = Y_test[:]
df['reco'] = mypreds.flatten()[:]
df['basic'] = basic
df = pd.DataFrame.from_dict(df)

df.eval('rel_res = (reco)/gen',inplace=True)
df.eval('rel_res_basic = basic/gen',inplace=True)

df.head()

In [ ]:
from scipy import stats
import pandas as pd

df = {}
df['gen'] = Y_test[:]
df['reco'] = mypreds.flatten()[:]
df['basic'] = basic
df = pd.DataFrame.from_dict(df)

df.eval('rel_res = (reco)/gen',inplace=True)
df.eval('rel_res_basic = basic/gen',inplace=True)
df.head()
fig = plt.figure( figsize=(8, 6))
temp = df.groupby(pd.cut(df['gen'], bins=np.geomspace(1,80,12)))['rel_res']
trim_mean = temp.apply(stats.trim_mean, 0.05)
trim_std = temp.apply(stats.mstats.trimmed_std, limits=(0.05,0.05))
temp = temp.agg(['mean', 'std', 'size']).reset_index()
x = [i.mid for i in temp['gen']]
print (x)
plt.plot(x,temp['std']/temp['mean'],'o-',label='STD')
plt.plot(x, trim_std/trim_mean,'o-',label='90% STD')











plt.xlabel('Generated energy [GeV]',fontsize=22)

plt.legend(fontsize=22)
plt.grid()
plt.ylabel('Relative energy resolution')
plt.ylim([0.0,0.45])

def stochastic(a,b):
    return a/np.sqrt(b) +0.034

#energy = np.linspace(1,10,100)
#plt.plot(energy, stochastic(0.34,energy))
plt.tight_layout()
plt.show()




In [ ]:
#Binning
N = 40
E_Max = 100
E_Bins = np.linspace(0,E_Max,N+1)

#Goal: slices defined by bin of truthE, filled with prediction distributions
indecies = np.digitize(Y_test,E_Bins)
max_count = ((np.bincount(indecies).max()))
slices = np.empty((N,max_count))
slices.fill(np.nan)

counter = np.zeros(N,int)
avg_truth = np.zeros(N,float)

for pred,bin,truth in zip(mypreds,indecies,Y_test):
    slices[bin-1][counter[bin-1]] = pred
    counter[bin-1]+=1
    avg_truth[bin-1]+=truth

#Resoluton: stdev(pred)/avg_truth    
avg_truth = avg_truth/counter
stdev_pred = np.nanstd(slices,axis=1)
resolution = stdev_pred/avg_truth

In [ ]:
fig=plt.figure(figsize=(14,10))
plt.title("PFN reconstruction",fontsize=25)
plt.ylabel("$(\sigma_{E,\mathrm{Pred}}/E_\mathrm{Truth})$",fontsize=24)
plt.xlabel("$E_\mathrm{Truth}$ [GeV]",fontsize=24)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tick_params(direction='in',right=True,top=True,length=10)
#plt.ylim(-0.02,0.4)
#plt.ylim(0,0.22)
plt.xlim(-0.01,100.01)
errors = 1.0/(np.sqrt(2*counter-2))*stdev_pred
ax = plt.subplot(1,1,1)
first_bin = 0
last_bin = N
#plt.errorbar(avg_truth[first_bin:last_bin],resolution[first_bin:last_bin],yerr=errors[first_bin:last_bin],
#             linestyle="-",linewidth=2.0,capsize=4,capthick=1.2,elinewidth=1.2,ecolor='black',marker="o",color='dodgerblue',alpha=0.7)

plt.errorbar(avg_truth[first_bin:last_bin],resolution[first_bin:last_bin],
             linestyle="-",linewidth=2.0,capsize=4,capthick=1.2,elinewidth=1.2,ecolor='black',marker="o",color='dodgerblue',alpha=0.7)

_ = plt.text(0.7,0.93,"Stat. Error: $\dfrac{\sigma}{\sqrt{2N-2} } $",transform=ax.transAxes,fontsize=20)
plt.savefig("resolution_plot.pdf")

In [ ]:
fig,axs = plt.subplots(int(N/10),10, figsize=(32, 16),sharex=False,sharey=True,constrained_layout=True)
for i in range(N):
    row = int(i/10)
    col = i%10
    ax = axs[row,col]
    if (col==0):
        ax.set_ylabel("Counts",fontsize=15)
    #temp_bin = np.linspace(avg_truth[i]-2.0,avg_truth[i]+2.0,16)
    ax.set_title("%1.1f $ < E_\mathrm{Truth} < $%1.1f [GeV]"%(E_Bins[i],E_Bins[i+1]))
    ax.set_xlabel("Predicted Eenergy")
    ax.hist(slices[i],range=(0,100),bins=40,label="Predicted Energies")
    ax.axvline(x=avg_truth[i],color='red',alpha=0.3,linestyle="--",label="Avg. $E_\mathrm{Truth} = %1.1f$"%(avg_truth[i]))
    ax.legend(fontsize=7.5)
    ax.tick_params(direction='in',right=True,top=True,length=5)
plt.savefig("resolutions_slices.pdf")

In [ ]:
from copy import copy
from matplotlib.colors import LogNorm
fig, axes = plt.subplots(nrows=1, figsize=(14, 10), constrained_layout=True)
cmap = copy(plt.cm.plasma)
cmap.set_bad(cmap(0))
edges=np.linspace(-10,110,121)
h, xedges, yedges = np.histogram2d(Y_test, mypreds[:,0], bins=[edges, edges])
#xedges=yedges
pcm = axes.pcolormesh(xedges, yedges, h.T, cmap=cmap,
                         norm=LogNorm(vmin=1.0e-2,vmax=2.0e3), rasterized=True)
cb = fig.colorbar(pcm, ax=axes, pad=0)
cb.set_label("Counts",fontsize=22)
cb.ax.tick_params(labelsize=20)
axes.set_xlabel("Generated Energy",fontsize=22)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.ylim(-10,110)
plt.xlim(-10,110)
axes.set_ylabel("Predicted Energy",fontsize=25)
axes.set_title("Predicted vs. Generated Energy",fontsize=30)
plt.savefig("Gen_vs_Pred.pdf")
print(np.size(yedges))

In [ ]:
#Find the peak and zoom in
fig = plt.figure(figsize=(18,5))
plt.suptitle("Predictions",fontsize=25)
ax = plt.subplot(1, 2, 1)
plt.xlabel("Predicted Energy [GeV]",fontsize=18)
bins = np.linspace(0,100,300)
freq = plt.hist(mypreds, bins=bins)

ax = plt.subplot(1, 2, 2)
plt.xlabel("Predicted Energy [GeV]",fontsize=18)
plt.text(0.03,0.9,"[Zoomed In]",transform=ax.transAxes,fontsize=18)
maxbin = np.argmax(freq[0])
zoom = np.linspace(bins[maxbin]-0.01,bins[maxbin]+0.01,100)
mask = np.where(np.logical_and(mypreds>=bins[maxbin]-0.01, mypreds<=bins[maxbin]+0.01))[0]
_ = plt.hist(mypreds[mask],alpha=.3,color="black",bins=zoom)
print("%i / %i Events"%(len(mask),len(mypreds)),"[{:.3%}]".format( (len(mask)/len(mypreds)) ) )

In [ ]:
cm = plt.cm.get_cmap('plasma')
cell_vars = ["Energy","Cell X","Cell Y","Cell Z","Layer 1 Position", "Layer 2 Position"]
bins = [np.linspace(0.01,500,200),np.linspace(-500,500,100),
        np.linspace(-500,1300,100),np.linspace(-1.5,2.5,100),
        np.linspace(-275,350,100),np.linspace(-275,350,100)]
weird_data=X_test[mask]
fig = plt.figure(figsize=(18,9))
#plt.subplots_adjust(left=None, bottom=1, right=None, top=1.5, wspace=None, hspace=None)
for i in range(images.shape[1]-1):
    ax = plt.subplot(2, 3, i+1)
    plt.scatter(np.ravel(weird_data[:,i,:]),np.ravel(weird_data[:,i+1,:]),s=2)
    plt.xlabel(cell_vars[i])
    plt.ylabel(cell_vars[i+1])
    #plt.title("%s"%(cell_vars[i]),fontsize=20)
    plt.suptitle("Normalized Cell Scatter Plots",fontsize=25)
plt.savefig("weird_Cell_Data.pdf")